# Making a playable chess game with numpy and python

In [1]:
import numpy as np

In [2]:
#We shall represent the board as an 8x8 numpy array
Board = np.zeros((8, 8),  dtype=np.int) #np.int removes decimal point
Board

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [3]:
"""
We shall represent the pieces by a 2 digit number:

The first number repsents the piece type.
Pawn = 1
Rook = 2
Knight = 3
Bishop = 4
Queen = 5
King = 6

The second represents if the piece is black or white. 
0 = White
1 = Black


e.g 
10 - is a white Pawn.
61 - is a black King
"""

'\nWe shall represent the pieces by a 2 digit number:\n\nThe first number repsents the piece type.\nPawn = 1\nRook = 2\nKnight = 3\nBishop = 4\nQueen = 5\nKing = 6\n\nThe second represents if the piece is black or white. \n0 = White\n1 = Black\n\n\ne.g \n10 - is a white Pawn.\n61 - is a black King\n'

In [6]:
# Now we specify the starting position 


#Blacks pieces 
Board[1][0:] = 11
Board[0][0] = 21
Board[0][1] = 31
Board[0][2] = 41
Board[0][3] = 51
Board[0][4] = 61
Board[0][5] = 41
Board[0][6] = 31
Board[0][7] = 21

#Whitess pieces 
Board[6][0:] = 10
Board[7][0] = 20
Board[7][1] = 30
Board[7][2] = 40
Board[7][3] = 50
Board[7][4] = 60
Board[7][5] = 40
Board[7][6] = 30
Board[7][7] = 20

In [7]:
Board

array([[21, 31, 41, 51, 61, 41, 31, 21],
       [11, 11, 11, 11, 11, 11, 11, 11],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [10, 10, 10, 10, 10, 10, 10, 10],
       [20, 30, 40, 50, 60, 40, 30, 20]])

In [ ]:
Board[4][1] = 101

In [ ]:
Board

# Here we stop nonsense moves


In [ ]:
#Here we create a function to check input values are allowable
numbers = [str(i) for i in range(8)]
Allowable_squares = []
for i in numbers:
    for j in numbers:
        Allowable_squares.append(i+j)
        
def allowable_input(initial_square, target_square):
    if initial_square not in Allowable_squares:
        return False
    if target_square not in Allowable_squares:
        return False
    else:
        return True

In [ ]:
#Checking a piece has been selected
def piece_selected(initial_square, target_square, chess_position):
    if chess_position[int(initial_square[0])][int(initial_square[1])] == 0:
        return False
    else:
        return True  

In [ ]:
#checking player isnt attempting to take their own piece
def no_self_sacrifice(initial_square, target_square, chess_position):
    if chess_position[int(target_square[0])][int(target_square[1])] ==0:
        return True
    else:
        initial_square_colour = str(chess_position[int(initial_square[0])][int(initial_square[1])])[1]
        target_square_colour = str(chess_position[int(target_square[0])][int(target_square[1])])[1]
    
        if initial_square_colour == target_square_colour:
            return False
        else:
            return True

# We we start to define how pieces move

In [ ]:
"""
I shall Group pieces together from easiest to hardest to write, depending on how they move.


GROUP 1: Depends only on immediate position, and other pieces do no influence how it moves.

- Knight, although the knight jumps around the board, it has no special rules, indeed it can move to any square
as long as not occupied by one of its own. 

- Bishop, Queen - These pieces also have no special rules however, they are contrained by their own pieces as they can't
jump unlie the knight

GROUP 2: Depends on the move before:

- Pawn - Be wary of en passant! - Also have to consider promotions

GROUP 3: Depends on how pieces have moved throughout entire game.

- King, Rook - These pieces can move in special ways via castling, both king and queenside, so we have to 
create functions that allow us to check if castling is legal, on either side: 1) King and Knight can't have already move. 
2)You can't castle into or through check. It seems best to write these pieces last as these moves depend and can be 
influnced by every other piece. Note, movement rules do not depend on position but rather if thay have already moved.

The King is the most complex pieces.
"""

In [ ]:
#To define how pieces move, i shall write a function that generates an array of legal squares


In [ ]:
"""
In essence whatever the starting position of white, we either add or subtract 1 or 2 to each square value
If any are not on (0,7) then we remove them as they are not on the board
If they land on their own piece its not allowed
"""
def possible_knight_positions(initial_square,position): #Note enter initial_square as a str
    
    xlabel = int(initial_square[0]) 
    ylabel =int(initial_square[1]) 
    
    #print(xlabel)
    #print(ylabel)
    
    #Probably could be made cleaner later
    x_addition = np.array([-2,-2,-1,-1,1,1,2,2])
    y_addition = np.array([1,-1,2,-2,2,-2,1,-1])
    possible_x = x_addition + xlabel
    possible_y = y_addition + ylabel
    
    tuple_of_possible_positions = list(zip(possible_x,possible_y))
    
    
    #Removing tuples not on the board
    acceptable_values = [0,1,2,3,4,5,6,7]
    result1 = [i for i in tuple_of_possible_positions if i[0] in acceptable_values]
    """
In essence whatever the starting position of white, we either add or subtract 1 or 2 to each square value
If any are not on (0,7) then we remove them as they are not on the board
If they land on their own piece its not allowed
"""
def possible_knight_positions(initial_square,position): #Note enter initial_square as a str
    
    xlabel = int(initial_square[0]) 
    ylabel =int(initial_square[1]) 
    
    #print(xlabel)
    #print(ylabel)
    
    #Possible knights mo
    x_addition = np.array([-2,-2,-1,-1,1,1,2,2])
    y_addition = np.array([1,-1,2,-2,2,-2,1,-1])
    possible_x = x_addition + xlabel
    possible_y = y_addition + ylabel
    
    tuple_of_possible_positions = list(zip(possible_x,possible_y))
    
    
    #Removing tuples not on the board
    acceptable_values = [0,1,2,3,4,5,6,7]
    result1 = [i for i in tuple_of_possible_positions if i[0] in acceptable_values]
    result2 = [i for i in result1 if i[1] in acceptable_values]
    
     #Removing tuples that represent taking own piece
    for i in 
    
    
    
    
    
    
    
    #Now we remove elements from tuple_of_possible_positions if they contain numbers not in 0 - 7
    #for index, tuples in enumerate(tuple_of_possible_positions):
       # xlabel, ylabel = tuples[0], tuples[1]
       # if xlabel not in [0,1,2,3,4,5,6,7]:
        #    tuple_of_possible_positions.pop(index)
            
       # if ylabel not in [0,1,2,3,4,5,6,7]:
         #   tuple_of_possible_positions.pop(index)
    
   # print(tuple_of_possible_positions)
     
     = [i for i in result1 if i[1] in acceptable_values]
    
   
    
    
    
    
    
    #Now we remove elements from tuple_of_possible_positions if they contain numbers not in 0 - 7
    #for index, tuples in enumerate(tuple_of_possible_positions):
       # xlabel, ylabel = tuples[0], tuples[1]
       # if xlabel not in [0,1,2,3,4,5,6,7]:
        #    tuple_of_possible_positions.pop(index)
            
       # if ylabel not in [0,1,2,3,4,5,6,7]:
         #   tuple_of_possible_positions.pop(index)
    
   # print(tuple_of_possible_positions)
     
    

In [88]:
possible_knight_positions('71',Board)

[(5, 2), (5, 0), (6, 3), (6, -1), (8, 3), (8, -1), (9, 2), (9, 0)]
[(5, 2), (5, 0), (6, 3)]


In [69]:
mylist = [(1,2),(3,4),(4,6),(7,8)]
for j,k in mylist:
    if j or k not in [0,1,2,3,4,5,6,7]:
        my

NameError: name 'my' is not defined

In [9]:
def myfunc(a,b):
    list_of_possible = []
    values = [-2,-1,1,2]
    
    

TypeError: 'int' object is not subscriptable

In [ ]:
def possible_bishop_positions(initial_square,position):
    

In [ ]:
def possible_queen_positions(initial_square,position):

In [ ]:
def possible_pawn_positions(initial_square,position, enpassant, Promotion = 'Queen'):
    

In [ ]:
def possible_rook_positions(initial_square, position, Castle_avaible_kingside, Castle_avaible_queenside):

In [ ]:
def possible_King_positions(initial_square, position, Castle_avaible_kingside, Castle_avaible_queenside):
    

In [ ]:
#First we write a function to alter our Board
def move_a_piece(initial_square, target_square, chess_position): #Enter a digit number corresponding to our Board array
    if initial_square not in Allowable_squares:
        print('target_square format incorrect')
    elif target_square not in Allowable_squares:
        print('target_square format incorrect')
    else:
        initial_square_index1, initial_square_index2 = int(initial_square[0]) , int(initial_square[1])
        target_square_index1, target_square_index2 = int(target_square[0]) , int(target_square[1])
        print(chess_position[initial_square_index1],chess_position[initial_square_index1])
    

In [ ]:
'01'[0]

In [ ]:
move_a_piece('01', '02')
    

In [ ]:
game_over = False
turn ==0:
while not game_over:
    if turn == 0:
        move == input()
    
    if turn == 0:
        